#### GROMACS for CHARMM-GUI
(See also: [`GROMACS_for_production.ipynb`](https://colab.research.google.com/github/bioinfkaustin/gromacs-on-colab/blob/main/GROMACS_for_production.ipynb).)

#### Documentation
Please click ***↳ cells hidden*** below to show the documentation for this notebook.

##### About this software

> This notebook processes a **CHARMM-GUI system archive** (`.tgz`), producing a **GROMACS-ready folder for production runs**.
>
> A protein system prepared with the CHARMM-GUI **Solution Builder** or **Membrane Builder** must be provided. Optionally, a docked ligand conformation prepared with **Ligand Reader** may be provided, in which case the two structures and topologies will be merged into a protein-ligand complex. If the system spans a membrane, $z$-axis walls may be defined if desired.
>
> The recommended **minimisation** and **equilibration** simulations are then run with **GROMACS**, which automatically utilises the GPU if one is allocated. The equilibrated system is saved for a later production simulation. 

##### License

> This notebook as a work of software is licensed under the terms of the [AGPL-3.0](https://opensource.org/licenses/AGPL-3.0) or later.
>
> However, this notebook downloads and runs Chimera, which is free only for non-commercial use. Please do not run this notebook if doing so would violate the [UCSF Chimera Non-Commercial Software License Agreement](https://www.cgl.ucsf.edu/chimera/license.html).

#### Configuration

In [ ]:
import os
import re
import shutil

#@markdown Provide the location of the `.tgz` from **Solution Builder** or **Membrane Builder**. Optionally, you may also include a docked ligand, as a `.tgz` from **Ligand Reader**. (If you do not wish to use Google Drive, then in the left pane, go to *Files*, then *Upload*.)
protein_archive = "{GoogleDrive}/CHARMM-GUI/7FBF_FABPH.tgz" #@param {type: "string"}
ligand_archive = "{GoogleDrive}/CHARMM-GUI/7FBF_octanoic_acid.tgz" #@param {type: "string"}
# defaults: {GoogleDrive}/CHARMM-GUI/7FBF_FABPH.tgz
#           {GoogleDrive}/CHARMM-GUI/7FBF_octanoic_acid.tgz

#@markdown If the system contains a lipid membrane, provide the names of the lipid residues. (For example, "`POPC CHL1`".)
lipid_residue_names = "" #@param {type: "string"}

#@markdown Instead of a periodic boundary along the $z$-axis, use walls. (This is slow and unphysical!) Intended for membrane systems, i.e. only the top and bottom are walled. 
add_walls = False #@param {type: "boolean"}

#@markdown Specify a new folder in which to save the equilibrated output -- the production simulation can then be run in this folder.
output_folder = "{GoogleDrive}/GROMACS/7FBF_FABPH_vs_octanoic_acid" #@param {type: "string"}
# default: {GoogleDrive}/GROMACS/7FBF_FABPH_vs_octanoic_acid

#@markdown **After filling in this form, run the notebook by clicking *Runtime -> Run all* in the toolbar.**


#
# Make sure that the notebook is in the start directory
#

if "START" not in os.environ or not os.environ["START"]:
  %env START={os.getcwd()}
else:
  %cd {os.environ["START"]}


#
# Validate the input values
#


def google_drive_format(folder):
  if "{GoogleDrive}" in folder:
    if not folder.startswith("{GoogleDrive}"):
      raise ValueError(f"Error: {{GoogleDrive}} is a path prefix, but appears later: {folder}")
    if not os.path.isdir("/content/drive/MyDrive"):
      from google.colab import drive
      drive.mount("/content/drive")
  return folder.format(GoogleDrive="/content/drive/MyDrive")
  #             ^^^ raises KeyError if any {...} placeholder is present except {GoogleDrive}


# protein_archive

protein_archive = os.path.abspath(google_drive_format(protein_archive.strip()))

if not os.path.isfile(protein_archive):
  raise RuntimeError(f"Error: protein archive not found: {protein_archive}")

archive_exts = [".tar.gz", ".tgz"]
if not any(protein_archive.endswith(ext) for ext in archive_exts):
  raise RuntimeError(f"Error: protein archive should be a .tgz file, but got: {protein_archive}")


# ligand_archive

ligand_archive = os.path.abspath(google_drive_format(ligand_archive.strip()))

if ligand_archive and not os.path.isfile(ligand_archive):
  raise RuntimeError(f"Error: expecting ligand archive, but not found: {ligand_archive}")

if ligand_archive and not any(ligand_archive.endswith(ext) for ext in archive_exts):
  raise RuntimeError(f"Error: ligand archive should be a .tgz file, but got: {ligand_archive}")


# lipid_residue_names

lipid_residue_names = lipid_residue_names.strip()
if re.match(r"^[0-9a-zA-Z ]*$", lipid_residue_names) is None:
  raise RuntimeError(f"Error: expecting residue names to be blank or space separated residues, but got: {lipid_residue_names}")


# add_walls

add_walls_bash = "true" if add_walls else "false"


# output_folder

output_folder = os.path.abspath(google_drive_format(output_folder.strip()))

os.makedirs(output_folder, exist_ok=True)

if os.path.isfile(output_folder + "/conf.gro"):
  raise RuntimeError(f"Error: expecting empty folder, but found existing output: {output_folder}")


#
# Use a clean scratch directory for the rest of the run
#

try:
  shutil.rmtree("scratch")
except FileNotFoundError:
  pass
os.makedirs("scratch")
%cd "scratch"

#### Input

In [ ]:
%%bash -s "$protein_archive"
protein_archive="$1"
#@markdown Extract the system from the protein archive. This should come from either **Solution Builder** or **Membrane Builder**.

tar -xzf "$protein_archive"
mv charmm-gui-*/ "solution_builder"

In [ ]:
%%bash -s "$ligand_archive"
ligand_archive="$1"
#@markdown Extract the docked ligand from the ligand archive, if one was given.
#@markdown
#@markdown **`/!\`** This file **must** come from putting the **docking output `conf.mol2`** into **Ligand Reader**. **`/!\`**
#@markdown
#@markdown (Using the docking output is the only way the docked pose coordinates can go into CHARMM-GUI and then into GROMACS.)

if [[ -z "$ligand_archive" ]]; then
  exit 0
fi

tar -xzf "$ligand_archive"
mv charmm-gui-*/ "ligand_reader"

#### Installation

In [ ]:
%%bash
#@markdown When this cell runs, **GROMACS** is loaded from Google Drive. If not found, it is downloaded and compiled from source code. (This takes a while.)

if [[ -d "/usr/local/gromacs" ]]; then
  exit 0  # already installed
fi

gromacs_version="gromacs-2023-rc1" #@param {type: "string"}
usr_local_gromacs="/content/drive/MyDrive/usr_local_${gromacs_version}.tar.gz"

export DEBIAN_FRONTEND=noninteractive
add-apt-repository -y "ppa:ubuntu-toolchain-r/test"
apt-get update
apt-get -y install --no-install-recommends "g++-9"
update-alternatives --install "/usr/bin/gcc" "gcc" "/usr/bin/gcc-9" 99
update-alternatives --install "/usr/bin/g++" "g++" "/usr/bin/g++-9" 99

if [[ -s "${usr_local_gromacs}" ]]; then
  tar -xzf "${usr_local_gromacs}" -C "/usr/local"
else
  wget "ftp://ftp.gromacs.org/gromacs/${gromacs_version}.tar.gz"
  tar -xzf "${gromacs_version}.tar.gz"
  cd "${gromacs_version}"
  mkdir "build"
  cd "build"
  cmake .. -DGMX_BUILD_OWN_FFTW=ON -DGMX_GPU=CUDA
  make -j $(nproc)
  make install # -> /usr/local/gromacs

  if [[ -d "$(dirname "$usr_local_gromacs")" ]]; then
    tar -czf "usr_local_gromacs.tar.gz" -C "/usr/local" "gromacs"
    mv "usr_local_gromacs.tar.gz" "${usr_local_gromacs}"
  fi
fi

In [ ]:
%%bash
#@markdown **Chimera** is loaded from Google Drive. If not found, the latest headless Linux version is downloaded and installed.

if [[ ! -d "ligand_reader" ]]; then
  exit 0  # Chimera is only used to manipulate ligands, so no need for it if no ligand given
fi

if [[ -x "$START/UCSF-Chimera64/bin/chimera" ]]; then
  exit 0  # already installed
fi

chimera_osmesa_bin="/content/drive/MyDrive/chimera-alpha-linux_x86_64_osmesa.bin"

if [[ -s "${chimera_osmesa_bin}" ]]; then
  cp "${chimera_osmesa_bin}" .
  chmod +x "chimera-alpha-linux_x86_64_osmesa.bin"
else
  curl \
    -X "POST" \
    -F "file=alpha/chimera-alpha-linux_x86_64_osmesa.bin" \
    -F "choice=Accept" \
    -s "https://www.cgl.ucsf.edu/chimera/cgi-bin/secure/chimera-get.py" > "refresh.tmp"

  req=$(fgrep 'meta http-equiv="Refresh"' "refresh.tmp" | sed 's|^<meta http-equiv="Refresh" content="2;url=||; s|">$||')
  curl -s "https://www.cgl.ucsf.edu/$req" > "chimera-alpha-linux_x86_64_osmesa.bin"
  rm "refresh.tmp"

  chmod +x "chimera-alpha-linux_x86_64_osmesa.bin"
  if [[ -d "$(dirname "$chimera_osmesa_bin")" ]]; then
    cp "chimera-alpha-linux_x86_64_osmesa.bin" "${chimera_osmesa_bin}"
  fi
fi

./chimera-alpha-linux_x86_64_osmesa.bin < <(echo "$START/UCSF-Chimera64"; echo "0"; echo "")
# -> ./UCSF-Chimera64

In [ ]:
#@markdown The `$PATH` envvar is modified to include Chimera.

chimera_bin = os.environ["START"] + "/UCSF-Chimera64/bin"
if os.path.isdir(chimera_bin) and chimera_bin not in os.environ["PATH"]:
  %env PATH={chimera_bin}:{os.environ["PATH"]}

In [ ]:
%%bash
#@markdown The utility **`cgenff_charmm2gmx_py3_nx2.py`** is downloaded and installed.

if [[ -s "ligand_reader/gromacs/LIG.itp" ]]; then
  exit 0  # no need to generate LIG.itp, CHARMM-GUI already took care of it
fi

if [[ -x "$START/cgenff_charmm2gmx_py3_nx2.py" ]]; then
  exit 0  # already installed
fi

if [[ ! -d "$START/charmm36.ff" ]]; then
  charmm_vers="charmm36-jul2022" #@param {type: "string"}
  wget -O "$charmm_vers.ff.tgz" "https://mackerell.umaryland.edu/download.php?filename=CHARMM_ff_params_files/$charmm_vers.ff.tgz"
  tar -xzf "$charmm_vers.ff.tgz"
  mv "$charmm_vers.ff" "$START/charmm36.ff"
fi

if [[ ! -d "$START/miniconda" ]]; then
  miniconda_vers="Miniconda3-py38_22.11.1-1-Linux-x86_64" #@param {type: "string"}
  wget "https://repo.anaconda.com/miniconda/$miniconda_vers.sh"
  bash "$miniconda_vers.sh" -b -p "$START/miniconda3"
  eval "$("$START/miniconda3/bin/conda" shell.bash hook)"
  conda create --name "charmm2gmx" --yes "python<=3.8" "networkx=2.3" "numpy<=1.23"
fi

if [[ ! -x "$START/miniconda3/envs/charmm2gmx/bin/cgenff_charmm2gmx_py3_nx2.py" ]]; then
  wget -O "cgenff_charmm2gmx_py3_nx2.py" "https://mackerell.umaryland.edu/download.php?filename=CHARMM_ff_params_files/cgenff_charmm2gmx_py3_nx2.py"
  sed -i"" 's/\r//g' "cgenff_charmm2gmx_py3_nx2.py"
  chmod +x "cgenff_charmm2gmx_py3_nx2.py"
  mv "cgenff_charmm2gmx_py3_nx2.py" "$START/miniconda3/envs/charmm2gmx/bin/"
fi

#### Processing

In [ ]:
%%bash
#@markdown The GROMACS version of the protein structure is **combined** with a GROMACS version of a docked ligand, and any waters overlapping with the ligand are deleted. 

if [[ ! -d "ligand_reader" ]]; then
  exit 0
fi

source "/usr/local/gromacs/bin/GMXRC.bash"

# Combine the protein and ligand, deleting overlapping waters
# Makes "combination.gro"

#                                vvv step3 in Solution Builder, step5 in Membrane Builder
chimera solution_builder/gromacs/step*_input.pdb "ligand_reader/ligandrm.pdb" "solution_builder/step1_pdbreader.pdb" <<EOF
matchmaker #0 #2
matrixcopy #2 #1
write relative #0 #1 LIG.pdb
zonesel #1 1.5 #0:TIP3
select up
write selected relative #0 #0 deleted_TIP3.pdb
delete sel
combine #0,1
write relative #0 #3 combination.pdb
EOF

gmx editconf -f "combination.pdb" -o "conf.gro"

( head -n-1 "conf.gro"; tail -n-1 solution_builder/gromacs/step*_input.gro; ) > "sponge.tmp"
mv "sponge.tmp" "conf.gro"

In [ ]:
%%bash
#@markdown A GROMACS format **parameterisation** for the system is created.
#@markdown
#@markdown If a docked ligand conformation was provided, the generated topology includes the ligand molecule. If CHARMM-GUI did not output a GROMACS version of the ligand topology -- for instance if the ligand has lone pairs -- the tool `cgenff_charmm2gmx_py3_nx2.py` is used to generate the necessary topology files. Position restraints for the docked ligand are also generated.

# Update the topology files
# Makes "topol.top" and "toppar/

if [[ ! -d "ligand_reader" ]]; then

  # CHARMM-GUI generated topology suffices if we have not modified the system ourselves
  cp "solution_builder/gromacs/topol.top" .
  cp -r "solution_builder/gromacs/toppar" .

else

  source "/usr/local/gromacs/bin/GMXRC.bash"

  if [[ -s "ligand_reader/gromacs/LIG.itp" ]]; then
    
    # No lone pairs -- easy method

    cp -r "solution_builder/gromacs/toppar" .
    # Put the base forcefield files in the working folder to be combined
    mv "toppar/forcefield.itp" .
    cp "ligand_reader/gromacs/charmm36.itp" "forcefield_extras.itp"
    # Put the ligand file into the toppar/ folder
    cp "ligand_reader/gromacs/LIG.itp" "toppar/"

    easy_convert=true

  else

    # Lone pairs -- hard method
    
    eval "$("$START/miniconda3/bin/conda" shell.bash hook)"
    conda activate "charmm2gmx"

    chimera "ligand_reader/ligandrm.pdb" <<EOF
setattr m name LIG #0
write format mol2 #0 LIG.mol2
EOF
    # Expected ".str" file format reverse engineered from cgenff_charmm2gmx_py3_nx2.py
    # (since it doesn't appear to be documented anywhere...)
    {
      echo "* For use with CGenFF version 4.6"
      cat "ligand_reader/lig/lig.rtf"
      echo "read para"
      cat "ligand_reader/lig/lig.prm"
    } > "lig.str"
    sed -i"" "s/ lig / LIG /" "lig.str"
    mv "lig.str" "LIG.str"
    cgenff_charmm2gmx_py3_nx2.py "LIG" "LIG.mol2" "LIG.str" "$START/charmm36.ff"

    cp -r "solution_builder/gromacs/toppar" .
    rm "toppar/forcefield.itp"
    cp -r "$START/charmm36.ff" "toppar/"
    cp "lig.prm" "toppar/LIG.prm"
    cp "lig.itp" "toppar/LIG.itp"

    easy_convert=false

  fi

  while IFS="" read -r l; do
    if [[ "$l" == '#include "toppar/forcefield.itp"' ]]; then
      if $easy_convert; then
        echo "$l"
      else
        echo '#include "toppar/charmm36.ff/forcefield.itp"'
        echo '#include "toppar/LIG.prm"'
      fi
    elif [[ "$l" == '#include "toppar/TIP3.itp"' ]]; then
      echo "$l"
      echo '#include "toppar/LIG.itp"'
    elif [[ "$l" =~ ^TIP3 ]]; then
      IFS=$'\t' read -r name num_original <<< "$l"
      num_deleted=$(cat deleted_TIP3.pdb | fgrep "OH2 TIP3" | wc -l)
      num_now=$(($num_original - $num_deleted))
      printf "TIP3  \t%12d\n" "$num_now"
      echo "LIG   "$'\t'"           1"
    else
      echo "$l"
    fi
  done < "solution_builder/gromacs/topol.top" > "topol.top"

  # Position restraints for LIG
  gmx genrestr -f "ligand_reader/ligandrm.pdb" -fc 9999 <<< "0"
  {
    echo ""
    echo "#ifdef POSRES"
    tail -n+3 "posre.itp" | sed "s/9999/POSRES_FC_LIG/g"
    echo "#endif"
    echo ""
  } >> "toppar/LIG.itp"

fi

In [ ]:
#@markdown Define a subroutine to combine multiple .itp format GROMACS forcefield files, taking care with directive order.
#@markdown 
#@markdown If there are protein and ligand subsets of the CHARMM forcefield which need to be merged, do so.

def combine_itp_files(filenames_in, filename_out):
  """Combines multiple .itp files, taking care with directive order"""

  def parse_itp_file(filename):
    """Parses a .itp file into a dict of lists of lines"""
    sections = [""]
    grouped_lines = [[]]
    with open(filename) as f:
      for l in f:
        if l.startswith("["):
          sections.append(l)
          grouped_lines.append(list())
        grouped_lines[-1].append(l)
    if grouped_lines[-1][-1] != "\n":
      grouped_lines[-1].append("\n") # formatting
    return sections, grouped_lines

  all_parsed_files = [parse_itp_file(filename) for filename in filenames_in]

  directive_order_hint = ["", "[ defaults ]\n", "[ atomtypes ]\n", "[ bondtypes ]\n", "[ pairtypes ]\n", "[ angletypes ]\n", "[ dihedraltypes ]\n", "[ constrainttypes ]\n", "[ nonbond_params ]\n"]

  def dump(out, lines):
    for l in lines:
      out.write(l)
  
  with open(filename_out, "w") as out:
    # Try to stick to the documented order where possible -- https://manual.gromacs.org/current/reference-manual/topologies/topology-file-formats.html
    for key in directive_order_hint:
      for sections, grouped_lines in all_parsed_files:
        for section, lines in zip(sections, grouped_lines):
          if section == key:
            dump(out, lines)
            if key == "[ defaults ]\n":
              break # don't print [ defaults ] multiple times
        else:
          continue
        break # breaks iff inner loop break

    # For other directives, just put them at the end in the order they appear in their respective .itp files
    for sections, grouped_lines in all_parsed_files:
      for section, lines in zip(sections, grouped_lines):
        if section not in directive_order_hint:
          dump(out, lines)

if os.path.isfile("forcefield.itp") and os.path.isfile("forcefield_extras.itp"):
  combine_itp_files(["forcefield.itp", "forcefield_extras.itp"], "toppar/forcefield.itp")

In [ ]:
%%bash -s "$lipid_residue_names"
lipid_residue_names="$1"
#@markdown An **index** is built describing which parts of the system are fluid (solvent + ions), complex (protein + cofactors + ligand), and optionally membrane (lipids).

if [[ ! -d "ligand_reader" ]]; then
  
  # If no ligand has been merged into the system, the index.ndx provided by CHARMM-GUI will suffice
  cp "solution_builder/gromacs/index.ndx" .

else

  source "/usr/local/gromacs/bin/GMXRC.bash"

  gmx make_ndx -f < /dev/null 2> /dev/null | fgrep "  : " > "index_autodetect.txt"
  nr=$(cat "index_autodetect.txt" | wc -l)

  # Which ions are present in the solvent?
  defn_fluid='"TIP3"'
  for ion in "CL" "CLA" "K" "POT" "NA" "SOD"; do
    if fgrep -q " $ion " "index_autodetect.txt"; then
      fluid_defn="$fluid | \"$ion\""
    fi
  done

  #
  # Note: groups are named SOLV, MEMB, SOLU, SOLU_MEMB so as to follow
  # the CHARMM-GUI convention and maintain compatibility with its
  # outputted .mdp configuration files.
  #

  # Commands to create fluid groups
  nr_fluid=$nr
  {
    echo "$defn_fluid"
    echo "name $nr_fluid SOLV"
  } > "index_commands.txt"

  if [[ -z "$lipid_residue_names" ]]; then

    # The groups not-fluid, fluid are sufficient to describe the whole system
    nr_not_fluid=$(($nr + 1))
    {
      echo '! "SOLV"'
      echo "name $nr_not_fluid SOLU"
    } >> "index_commands.txt"

  else

    # Command to create membrane group
    defn_lipid=""
    for residue in $lipid_residue_names; do
      defn_lipid="$defn_lipid${defn_lipid:+ | }\"$residue\""
    done
    nr_membrane=$(($nr + 1))
    {
      echo "$defn_lipid"
      echo "name $nr_membrane MEMB"
    } >> "index_commands.txt"

    # Command to create "everything else" group, which should be the protein and any bound ligand, cofactors, etc.
    nr_complex=$(($nr + 2))
    {
      echo '! "SOLV" & ! "MEMB"'
      echo "name $nr_complex SOLU"
    } >> "index_commands.txt"

    # Combine the complex and membrane groups 
    nr_complex_and_membrane=$(($nr + 3))
    {
      echo '"SOLU" | "MEMB"'
      echo "name $nr_complex_and_membrane SOLU_MEMB"
    } >> "index_commands.txt"

  fi

  echo "q" >> "index_commands.txt"

  gmx make_ndx -f < "index_commands.txt"

fi

In [ ]:
%%bash -s "$add_walls_bash"
add_walls="$1"
#@markdown The `.mdp` configuration files provided by CHARMM-GUI are collated. If the ligand or wall options are selected then relevant parameters are added.

# Collate the .mdp configuration files provided by CHARMM-GUI

step0=$(ls -v "solution_builder/gromacs/" | fgrep "minimization.mdp")
cp "solution_builder/gromacs/$step0" "pre_0.mdp" 

i=1
while read -r step; do
  cp "solution_builder/gromacs/$step" "pre_$i.mdp"
  i=$(($i + 1))
done < <(ls -v "solution_builder/gromacs/" | fgrep "equilibration.mdp")


# Add configuration options for the ligand if present

if [[ -d "ligand_reader" ]]; then
  for f in pre_*.mdp; do
    bb="$(head -n1 "$f" | egrep -o "\-DPOSRES_FC_BB=[^ ]+")"
    sed -i"" "s/$bb/$(echo "$bb" | sed 's/_BB/_LIG/') $bb/" "$f"
  done
fi


# Add configuration options for walls if selected

if $add_walls; then

  source "/usr/local/gromacs/bin/GMXRC.bash"

  read -r x y z etc < <(tail -n1 "conf.gro")
  z_add=$(perl -le "print($z + 0.4)")
  gmx editconf -f "conf.gro" -translate 0 0 0.2 -box $x $y $z_add
  mv "out.gro" "conf.gro"

  cat > "walls_mdp.txt" <<EOS

; walls
pbc = xy
nwall = 2
wall-atomtype = CA CA
wall-type = 12-6
EOS

  for f in pre_*.mdp; do
    cat "walls_mdp.txt" >> "$f"
  done

fi

#### Simulation

In [ ]:
%%writefile "run.bash"
output_folder="$1"
#@markdown Create a script to run the simulation.

cp conf.gro restraint.gro

if ! mkdir -p "$output_folder"; then
  echo "Error: invalid folder: $output_folder" >&2
  exit 1
fi

#
# At this point we have:
# conf.gro index.ndx restraint.gro pre_*.mdp topol.top toppar/
#
# The below is a reproduction of the CHARMM-GUI run script, converted to Bash.
#

source "/usr/local/gromacs/bin/GMXRC.bash"
export GMX_MAXCONSTRWARN=-1

echo "Notice: saving output to folder: $output_folder"
sleep 1

i=0
while [[ -s "pre_$i.mdp" ]]; do
  if (( $i == 0 )); then
    gmx grompp -f "pre_$i.mdp" -o "pre_$i.tpr" -c "conf.gro" -r "restraint.gro" -p "topol.top" -n "index.ndx" -maxwarn 999
    gmx mdrun -deffnm "pre_$i"
  else
    prev=$(($i - 1))
    gmx grompp -f "pre_$i.mdp" -o "pre_$i.tpr" -c "pre_$prev.gro" -r "restraint.gro" -p "topol.top" -n "index.ndx" -maxwarn 999
    gmx mdrun -v -stepout 1000 -deffnm "pre_$i"
  fi
  i=$(($i + 1))
done

#
# Now we can save the minimised and equilibrated system
#

i=$(($i - 1))

# Get the CHARMM-GUI suggested production .mdp
sim_mdp=$(ls -v "solution_builder/gromacs/" | fgrep "production.mdp")
sed "s/ = 50000/ = 20000/" "solution_builder/gromacs/$sim_mdp" > "grompp.mdp"
#                  ^^^ 25 frames per ns
if [[ -s "walls_mdp.txt" ]]; then
  cat "walls_mdp.txt" >> "grompp.mdp"
fi

cp "pre_$i.gro" "$output_folder/conf.gro"
cp -r "grompp.mdp" "index.ndx" "restraint.gro" "topol.top" "toppar" *.log "$output_folder/"

In [ ]:
#@markdown Execute the simulation script:
#@markdown 1. Run the **minimisation and equilibration** steps.
#@markdown 2. Save the output to the specified output folder, from which production simulations can be run.

!bash "run.bash" "$output_folder"
!sleep 10

In [ ]:
#@markdown Finally, disconnect the runtime. (This option is ignored if the output folder is not in your Google Drive.)
disconnect = True #@param {type: "boolean"}

if disconnect and output_folder.startswith("/content/drive/MyDrive/"):
  from google.colab import runtime
  runtime.unassign()